In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/train_data/train.parquet')
df = df.loc[df['type']!=0]

df['hour'] = df.ts % (60*60*24)
df['day'] = df.ts % (60*60*24*7)
print( df.shape )
df.head()

(16547894, 6)


,session,aid,ts,type,hour,day
6,0,1649869,1659369893,1,57893,403493
7,0,461689,1659369898,1,57898,403498
8,0,305831,1659370027,2,58027,403627
9,0,461689,1659370027,2,58027,403627
43,0,789245,1659710157,1,52557,138957


In [3]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique','hour':'mean','day':'mean'})
item_features.columns = ['buy_ratio','count_item','count_user','hour_mean','day_mean']

In [4]:
item_features.head()

,buy_ratio,count_item,count_user,hour_mean,day_mean
aid,,,,,
1,1.000000,1,1,56602.000000,575002.000000
3,1.185185,81,61,54025.481481,346292.148148
4,1.000000,5,5,55142.800000,193382.800000
8,1.000000,2,2,56586.000000,358986.000000
10,1.333333,3,2,65387.666667,295787.666667


In [5]:
item_features2 = df.groupby('aid').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std','hour_std','day_std']

f32 = ['buy_ratio_std','hour_std','day_std']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [6]:
item_features2.head()

,buy_ratio_std,hour_std,day_std
aid,,,
200362,0.707107,118.793938,118.793938
1671258,0.707107,33.941124,33.941124
168060,-1.000000,-1.000000,-1.000000
117952,-1.000000,-1.000000,-1.000000
160952,0.457160,18085.318359,165162.906250


In [7]:
item_features['repeat'] = item_features.count_item / item_features.count_user

f32 = ['buy_ratio','hour_mean','day_mean','repeat']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item','count_user']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [8]:
item_features = cudf.concat([item_features,item_features2],axis=1)

In [9]:
item_features.head()

,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std
aid,,,,,,,,,
1,1.000000,1,1,56602.000000,575002.000000,1.000000,-1.000000,-1.000000,-1.000000
3,1.185185,81,61,54025.480469,346292.156250,1.327869,0.390868,14256.288086,162612.312500
4,1.000000,5,5,55142.800781,193382.796875,1.000000,0.000000,22686.736328,75285.539062
8,1.000000,2,2,56586.000000,358986.000000,1.000000,0.000000,10384.570312,295085.562500
10,1.333333,3,2,65387.667969,295787.656250,1.500000,0.577350,4047.108643,53823.906250


In [10]:
item_features.dtypes

buy_ratio        float32
count_item         int32
count_user         int32
hour_mean        float32
day_mean         float32
repeat           float32
buy_ratio_std    float32
hour_std         float32
day_std          float32
dtype: object

In [11]:
COLS = [f'{x}11' for x in item_features.columns]
item_features.columns = COLS
print(COLS)

['buy_ratio11', 'count_item11', 'count_user11', 'hour_mean11', 'day_mean11', 'repeat11', 'buy_ratio_std11', 'hour_std11', 'day_std11']


In [12]:
item_features.to_parquet('../../data/item_user_features/item40.pqt')